In [123]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support import expected_conditions as EC

from concurrent.futures import ThreadPoolExecutor
import multiprocessing

from Functions.function import movies_for_category, categories, print_process, movies
from bs4 import BeautifulSoup
import requests



In [2]:
categorias = categories()

|---------------------Categorias capturadas correctamente.---------------------|
|-----------------------Categorias demoro 10.16 segundos-----------------------|


In [3]:

# Obtener la cantidad de núcleos disponibles en el sistema
num_nucleos = multiprocessing.cpu_count()

# Crear un diccionario para almacenar los resultados
resultados_dict = {}

categorias_comienzo = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies_for_category, categorias))
    
    # Asignar los resultados al diccionario
    for categoria, resultado in zip(categorias, resultados):
        resultados_dict[categoria] = resultado

peliculas_total = []

for x,i in resultados_dict.items():
    for e in i:
        peliculas_total.append(e)

peliculas_total = set(peliculas_total)

categorias_fin = time.time()

print_process(f"La extraccion total de las categorias tubo una demora de {round((categorias_fin-categorias_comienzo),2)} segundos")


|Todo el contenido de la categoria 'history' fue capturado correctamente, demoro 21.22 segundos|
|Todo el contenido de la categoria 'biography' fue capturado correctamente, demoro 21.57 segundos|
|Todo el contenido de la categoria 'fantasy' fue capturado correctamente, demoro 23.48 segundos|
|Todo el contenido de la categoria 'crime' fue capturado correctamente, demoro 23.85 segundos|
|Todo el contenido de la categoria 'adventure' fue capturado correctamente, demoro 24.04 segundos|
|Todo el contenido de la categoria 'family' fue capturado correctamente, demoro 24.56 segundos|
|Todo el contenido de la categoria 'animation' fue capturado correctamente, demoro 25.99 segundos|
|Todo el contenido de la categoria 'action' fue capturado correctamente, demoro 28.29 segundos|
|Todo el contenido de la categoria 'horror' fue capturado correctamente, demoro 37.06 segundos|
|Todo el contenido de la categoria 'musical' fue capturado correctamente, demoro 10.14 segundos|
|Todo el contenido de la cate

In [4]:
peliculas_total = [x for x in peliculas_total if "/content/" in x]

In [5]:
# Crear un diccionario para almacenar los resultados
todo = {"titulo": [],"idioma" : [], "subtitulos" : [], "ano": [], "duracion_minutos": [], "categorias": [], "sinopsis": [], "tipo": [], "link": []}

comienzo_movies = time.time()

with ThreadPoolExecutor(max_workers=num_nucleos) as executor:
    # Ejecutar la función para cada categoría en paralelo
    resultados = list(executor.map(movies, peliculas_total))

    try:
        for resultado in resultados:
            for clave, valor in resultado.items():
                todo[clave].append(valor)
    except:
        print()

fin_movies = time.time()

print_process(f"El proceso de captura de la informacion de todo el contenido fue exsitoso, y demoro {round((fin_movies - comienzo_movies),2)} segundos")

In [7]:
df = pd.DataFrame(todo)

In [19]:
df.to_csv("Bases_datos/Peliculas_Series_Completo_Respaldo.csv" , index=False)

In [87]:
peliculas = df.loc[df["tipo"] == "Pelicula"].reset_index(drop=True)
series = df.loc[df["tipo"] == "Serie"].reset_index(drop=True)

In [88]:
peliculas

,titulo,ano,duracion,categorias,descripcion,tipo,link
0,Indigo,2017,2 min,"Short, Fantasy, Mystery, Sci-Fi, Thriller, Eur...",Eric resists taking the pills leads to an extr...,Pelicula,https://filmzie.com/content/indigo
1,Life in the Hole,2017,73 min,Thriller,"In a warehouse in the middle of nowhere, two r...",Pelicula,https://filmzie.com/content/life-in-the-hole-2017
2,Me and My Moulton,2014,13 min,"Animation, Comedy, Short, European Production","When a girl asks for a bicycle, she receives s...",Pelicula,https://filmzie.com/content/me-and-my-moulton
3,Red Army Hooligans,2018,87 min,Crime,"Tony, leader of a London Hooligan firm, seeks ...",Pelicula,https://filmzie.com/content/red-army-hooligans...
4,Lost Nomads,2010,24 min,"Documentary, European Production",Six friends embark on an unforgettable ski exp...,Pelicula,https://filmzie.com/content/lost-nomads-2010
...,...,...,...,...,...,...,...
2199,Thud Pilots,2018,90 min,Documentary,Discover the gripping Vietnam Air War in a doc...,Pelicula,https://filmzie.com/content/thud-pilots-2018
2200,Laughter Has No Color,2022,55 min,Documentary,This documentary explores the first very Black...,Pelicula,https://filmzie.com/content/laughter-has-no-co...
2201,The Investors,2018,17 min,"Short, Comedy, Thriller",The morality shakedown continues in Hollywood ...,Pelicula,https://filmzie.com/content/the-investors-2018
2202,All I Want Is You,2023,71 min,Drama,After holding her man down on a long 4-year bi...,Pelicula,https://filmzie.com/content/all-i-want-is-you-...


In [89]:
# Limpieza de metadata peliculas

peliculas['categorias'] = peliculas['categorias'].str.split(', ')
all_categories = set([cat for sublist in peliculas['categorias'].tolist() for cat in sublist])

for category in all_categories:
    peliculas[category] = 0
    for index, row in peliculas.iterrows():
        if category in row['categorias']:
            peliculas.at[index, category] = 1
    
peliculas.drop(columns=['categorias'], inplace=True)

peliculas["duracion"] = peliculas["duracion"].str.replace(" min","").astype(int)


In [128]:
def search(soup,*arg):
    """
        Primero ingresar componente de bs4 a analizar.
        luego Ingresar los valores a buscar anidados de la siguiente manera
        primer componente : "div/class"
        segundo componente : "p/class"
        y asi sucesivamente.
    """
    lista = [x.split("/") for x in arg]
    
    alm = soup
    
    for x in lista:

        alm = alm.find(x[0], class_=x[1])
    
    return alm.text
    


def movies(url):
    for _ in range(2):  # Intentar dos veces
        try:
            # Realizar la solicitud GET a la página web
            response = requests.get(url)

            # Verificar si la solicitud fue exitosa (código de estado 200)
            if response.status_code == 200:
                # Obtener el contenido HTML de la página
                html_content = response.content

                # Crear un objeto BeautifulSoup
                soup = BeautifulSoup(html_content, "html.parser")

                # Crear un diccionario para almacenar los datos
                diccionario_extraccion = {}

                try:
                    try:
                        diccionario_extraccion["titulo"] = soup.find(class_="fw-bold title").text
                        diccionario_extraccion["idioma"] = search(soup,"div/audio fs-15 label margin-bottom-30","p/d-inline-block")
                        diccionario_extraccion["subtitulos"] = search(soup,"div/subtitle fs-15 label", "span/")
                        diccionario_extraccion["ano"] = soup.find(class_="year").text
                        diccionario_extraccion["duracion_minutos"] = soup.find(class_="duration").text
                        diccionario_extraccion["categorias"] = soup.find(class_="category").text
                        diccionario_extraccion["sinopsis"] = soup.find(class_="fs-18").text
                        diccionario_extraccion["tipo"] = "Pelicula"
                        diccionario_extraccion["portada"] = search(soup,"div/poster",)
                        diccionario_extraccion["link"] = url
                    except:
                        diccionario_extraccion["titulo"] = soup.find(class_="fw-bold title").text
                        diccionario_extraccion["idioma"] = search(soup,"div/audio fs-15 label margin-bottom-30","p/d-inline-block")
                        diccionario_extraccion["subtitulos"] = search(soup,"div/subtitle fs-15 label", "p/fw-bold")
                        diccionario_extraccion["ano"] = soup.find(class_="year").text
                        diccionario_extraccion["duracion_minutos"] = "0 min"
                        diccionario_extraccion["categorias"] = soup.find(class_="category").text
                        diccionario_extraccion["sinopsis"] = soup.find(class_="fs-18").text
                        diccionario_extraccion["tipo"] = "Serie"
                        diccionario_extraccion["link"] = url
                except Exception as e:
                    # Si hay algún error al extraer los datos, imprimir el error y continuar con la siguiente iteración
                    print(f"Error al extraer los datos de la url {url} en el intento {_+1}: {e}")
                    continue
                
                return diccionario_extraccion

            else:
                print(f"Error al cargar la url {url} en el intento {_+1}: {response.status_code}")
        except Exception as e:
            print(f"Se produjo un error durante el intento {_+1} de la solicitud o el análisis de la página {url}: {e}")

    # Si todas las iteraciones fallan, retornar None
    return None

In [131]:
movies("https://filmzie.com/content/indigo")

{'titulo': 'Indigo',
 'idioma': 'English',
 'subtitulos': 'Arabic, German, English, Spanish, French, Hindi, Croatian, Italian, Dutch, Polish, Portuguese, Russian, Chinese',
 'ano': '2017',
 'duracion_minutos': '2 min',
 'categorias': 'Short, Fantasy, Mystery, Sci-Fi, Thriller, European Production',
 'sinopsis': 'Eric resists taking the pills leads to an extraordinary journey.',
 'tipo': 'Pelicula',
 'link': 'https://filmzie.com/content/indigo'}

In [125]:
peliculas

,titulo,ano,duracion,descripcion,tipo,link,Animation,Mystery,European Production,Thriller,...,Family,Western,Fantasy,Comedy,Biography,Crime,Horror,Reality TV,Sci-Fi,Music
0,Indigo,2017,2,Eric resists taking the pills leads to an extr...,Pelicula,https://filmzie.com/content/indigo,0,1,1,1,...,0,0,1,0,0,0,0,0,1,0
1,Life in the Hole,2017,73,"In a warehouse in the middle of nowhere, two r...",Pelicula,https://filmzie.com/content/life-in-the-hole-2017,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Me and My Moulton,2014,13,"When a girl asks for a bicycle, she receives s...",Pelicula,https://filmzie.com/content/me-and-my-moulton,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,Red Army Hooligans,2018,87,"Tony, leader of a London Hooligan firm, seeks ...",Pelicula,https://filmzie.com/content/red-army-hooligans...,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Lost Nomads,2010,24,Six friends embark on an unforgettable ski exp...,Pelicula,https://filmzie.com/content/lost-nomads-2010,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,Thud Pilots,2018,90,Discover the gripping Vietnam Air War in a doc...,Pelicula,https://filmzie.com/content/thud-pilots-2018,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2200,Laughter Has No Color,2022,55,This documentary explores the first very Black...,Pelicula,https://filmzie.com/content/laughter-has-no-co...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2201,The Investors,2018,17,The morality shakedown continues in Hollywood ...,Pelicula,https://filmzie.com/content/the-investors-2018,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2202,All I Want Is You,2023,71,After holding her man down on a long 4-year bi...,Pelicula,https://filmzie.com/content/all-i-want-is-you-...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
import pandas as pd
from datetime import time

# Crear un DataFrame de ejemplo
data = {'titulo': ['Indigo', 'Life in the Hole'],
        'ano': [2017, 2017],
        'duracion': ['2 min', '73 min']}
df = pd.DataFrame(data)

# Convertir la columna 'duracion' a valores de tipo datetime.time
df['duracion'] = df['duracion'].apply(lambda x: time(hour=int(x.split()[0])))

# Imprimir el DataFrame resultante
print(df)


ValueError: hour must be in 0..23

In [110]:
# Crea un objeto time con los segundos
tiempo_en_segundos = time.time(minute=valor_en_minutos)

# Convierte el objeto time a formato de hora
hora_formato = time.strftime('%H:%M', time.localtime(tiempo_en_segundos))


peliculas

,titulo,ano,duracion,descripcion,tipo,link,Animation,Mystery,European Production,Thriller,...,Family,Western,Fantasy,Comedy,Biography,Crime,Horror,Reality TV,Sci-Fi,Music
0,Indigo,2017,2,Eric resists taking the pills leads to an extr...,Pelicula,https://filmzie.com/content/indigo,0,1,1,1,...,0,0,1,0,0,0,0,0,1,0
1,Life in the Hole,2017,73,"In a warehouse in the middle of nowhere, two r...",Pelicula,https://filmzie.com/content/life-in-the-hole-2017,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Me and My Moulton,2014,13,"When a girl asks for a bicycle, she receives s...",Pelicula,https://filmzie.com/content/me-and-my-moulton,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
3,Red Army Hooligans,2018,87,"Tony, leader of a London Hooligan firm, seeks ...",Pelicula,https://filmzie.com/content/red-army-hooligans...,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Lost Nomads,2010,24,Six friends embark on an unforgettable ski exp...,Pelicula,https://filmzie.com/content/lost-nomads-2010,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,Thud Pilots,2018,90,Discover the gripping Vietnam Air War in a doc...,Pelicula,https://filmzie.com/content/thud-pilots-2018,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2200,Laughter Has No Color,2022,55,This documentary explores the first very Black...,Pelicula,https://filmzie.com/content/laughter-has-no-co...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2201,The Investors,2018,17,The morality shakedown continues in Hollywood ...,Pelicula,https://filmzie.com/content/the-investors-2018,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2202,All I Want Is You,2023,71,After holding her man down on a long 4-year bi...,Pelicula,https://filmzie.com/content/all-i-want-is-you-...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
peliculas.to_csv("Bases_datos/Peliculas.csv",index=False)

In [69]:
df.loc[df["tipo"] == "Serie"]

,titulo,ano,duracion,categorias,descripcion,tipo,link
38,The Ave,2015,0 min,Drama,Young entrepreneurs try to rise above the grit...,Serie,https://filmzie.com/content/the-ave-2015
45,Super Senses,2018,0 min,"Documentary, European Production",Have you ever wondered why some animals have s...,Serie,https://filmzie.com/content/super-senses-2018
52,We Are All Human,2020,0 min,"Documentary, Drama",A collection of short films that evolve around...,Serie,https://filmzie.com/content/we-are-all-human-2020
91,Conquering Northern China,2017,0 min,Documentary,Follow vloggers SerpentZA and Laowhy86 on a pu...,Serie,https://filmzie.com/content/conquering-norther...
121,Wilderness to Table with Chef Bri,2021,0 min,Reality TV,"Join Chef Bri on Wilderness to Table, the ulti...",Serie,https://filmzie.com/content/wilderness-to-tabl...
...,...,...,...,...,...,...,...
2203,The Inflight Food Trip,2019,0 min,Documentary,"Join Nik Loukas, the airline food guru, on a g...",Serie,https://filmzie.com/content/the-inflight-food-...
2219,Ghost Dimension Flying Solo,2017,0 min,"Horror, European Production",Join Sean as he investigates the UK's most hau...,Serie,https://filmzie.com/content/ghost-dimension-fl...
2251,37 Problems,2015,0 min,"Comedy, Drama",A 37-year-old screenwriter's focus on her care...,Serie,https://filmzie.com/content/37-problems-2015
2262,Heroes Manufactured: Creators Unleashed,2020,0 min,Reality TV,A reality series following North America's top...,Serie,https://filmzie.com/content/heroes-manufacture...
